In this notebook we'll explore topic modeling to discover broad themes in a collection of movie summaries.

In [1]:
import nltk
import re
import gensim
from gensim import corpora
import operator

nltk.download('stopwords')
from nltk.corpus import stopwords

import numpy as np
import random

random.seed(1)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dbamman/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
def read_stopwords(filename):
    stopwords={}
    with open(filename) as file:
        for line in file:
            stopwords[line.rstrip()]=1
    return stopwords

Since we're running topic modeling on texts with lots of names, we'll add the Jockers list of stopwords (which includes character names) to our stoplist.

In [3]:
stop_words = {k:1 for k in stopwords.words('english')}
stop_words.update(read_stopwords("../data/jockers.stopwords"))
stop_words["'s"]=1
stop_words=list(stop_words.keys())

In [4]:
def filter(word, stopwords):
    
    """ Function to exclude words from a text """
    
    # no stopwords
    if word in stopwords:
        return False
    
    # has to contain at least one letter
    if re.search("[A-Za-z]", word) is not None:
        return True
    
    return False

In [5]:
def read_docs(plotFile, metadataFile, stopwords):
    
    names={}
    box={}
    
    with open(metadataFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            name=cols[2]
            boxoffice=cols[4]
            if len(boxoffice) != 0:
                box[idd]=int(boxoffice)
                names[idd]=name
    
    n=5000
    target_movies={}


    sorted_box = sorted(box.items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_box[:n]:
        target_movies[k]=names[k]
    
    docs=[]
    names=[]
   
    with open(plotFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            text=cols[1]
            
            if idd in target_movies:
                tokens=nltk.word_tokenize(text.lower())
                tokens=[x for x in tokens if filter(x, stopwords)]
                docs.append(tokens)
                name=target_movies[idd]
                names.append(name)
    return docs, names

We'll read in summaries of the 5,000 movies with the highest box office revenues.

In [6]:
metadataFile="../data/movie.metadata.tsv"
plotFile="../data/plot_summaries.txt"
data, doc_names=read_docs(plotFile, metadataFile, stop_words)

We will convert the movie summaries into a bag-of-words representation using gensim's [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) methods.

In [7]:
# Create vocab from data; restrict vocab to only the top 10K terms that show up in at least 5 documents 
# and no more than 50% of all documents

dictionary = corpora.Dictionary(data)
dictionary.filter_extremes(no_below=5, no_above=.5, keep_n=10000)

In [8]:
# Replace dataset with numeric ids words in vocab (and exclude all other words)
corpus = [dictionary.doc2bow(text) for text in data]

In [9]:
num_topics=20

Now let's run a topic model on this data using gensim's built-in LDA.

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           passes=10,
                                           alpha='auto')

We can get a sense of what the topics are by printing the top 10 words with highest $P(word \mid topic)$ for each topic

In [11]:
for i in range(num_topics):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model.show_topic(i, topn=10)])))

topic 0:	miller shaw stu batman hood spirit gray dolls ring dent
topic 1:	car money train police truck town gets away find station
topic 2:	white mr. smith de snow curse mirror black magic apprentice
topic 3:	film kane company firm connor christmas stevens moore plant movie
topic 4:	fox dj rats rat angels finch hitler fairies malone bird
topic 5:	house room find goes finds night door away tells runs
topic 6:	war army soldiers men general orders mission military camp killed
topic 7:	father castle fight find village return battle named death city
topic 8:	duke dogs cooper team easy hayes sin dog fight caesar
topic 9:	doc red penguins west circus toys horse la machine klein
topic 10:	earth dr. find alien escape human kill power city killed
topic 11:	police murder case killed death car gun kill detective killer
topic 12:	race satan murphy church phantom beck cardinal pope axel warden
topic 13:	killer mrs. mask killed lake box kills kill collins find
topic 14:	band show music stage rock con

Another way of understanding topics is to print out the documents that have the highest topic representation -- i.e., for a given topic $k$, the documents with highest $P(topic=k | document)$.  How much do the documents listed here align with your understanding of the topics?

In [12]:
topic_model=lda_model 

topic_docs=[]
for i in range(num_topics):
    topic_docs.append({})
for doc_id in range(len(corpus)):
    doc_topics=topic_model.get_document_topics(corpus[doc_id])
    for topic_num, topic_prob in doc_topics:
        topic_docs[topic_num][doc_id]=topic_prob

for i in range(num_topics):
    print("%s\n" % ' '.join([term for term, freq in topic_model.show_topic(i, topn=10)]))
    sorted_x = sorted(topic_docs[i].items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x[:5]:
        print("%s\t%.3f\t%s" % (i,v,doc_names[k]))
    print()
    
    

miller shaw stu batman hood spirit gray dolls ring dent

0	0.889	Eyyvah Eyvah 2
0	0.294	Phone Booth
0	0.249	Batman Forever
0	0.243	The Manchurian Candidate
0	0.227	The Dark Knight

car money train police truck town gets away find station

1	0.926	Things Are Tough All Over
1	0.926	Redline
1	0.883	Rat Race
1	0.865	Money Train
1	0.767	Gone in Sixty Seconds

white mr. smith de snow curse mirror black magic apprentice

2	0.945	Mirror Mirror
2	0.721	What's Up, Doc?
2	0.523	Snow White and the Seven Dwarfs
2	0.509	Spring, Summer, Fall, Winter... and Spring
2	0.495	Harry Potter and the Deathly Hallows – Part 2

film kane company firm connor christmas stevens moore plant movie

3	0.717	What the #$*! Do We Know!?
3	0.584	U2 3D
3	0.565	Glee: The 3D Concert Movie
3	0.548	Nixon
3	0.536	A Mighty Heart

fox dj rats rat angels finch hitler fairies malone bird

4	0.579	A Midsummer Night's Dream
4	0.243	A Bug's Life
4	0.228	Sesame Street presents Follow That Bird
4	0.226	The Witches
4	0.212	The Tale of D